In [2]:
# 22.06.23 Bg Ausgabe mit +-Zeichen 
# 21.06.23 BG - Version 1.1. - Feldnamen sidn ale OK 
# 18.06.23 BG - amount fehlte als Feld 
# 15.06.23 BG - Version 1.0 - Fertig, Tabelle wird erfolgreich gefüllt 
# 15.06.23 BG - Neu gemacht, da irgend was weg geworfen 
# 12.06.23 BG - erster Wurf
#
# 
# verwendeter Report: Statment of Funds - Kapitalfluss Report 
#
# Infos zu dem Report 
# https://ibkrguides.com/reportingreference/reportguide/statement%20of%20fundsfq.htm
#



import locale
from xml.dom.minidom import parse
from datetime import datetime

import pyodbc  # Datenbankanbindung an Access 

locale.setlocale(locale.LC_ALL, 'de_DE')
# locale.setlocale(locale.decimal_point, ',' )
#print (locale.localeconv())


def insertStatements ( item , dbCursor , dbConn):
    
    #print ("-")
    #print ("Print Item --------- ")
    t = ";" # Definiere einen Trenner zum Lesen oder Excel 
    #t = " - " 

    transactionID = item.getAttribute ("transactionID") 
    if ( transactionID != ""):
        transactionID = int(transactionID )
    else:
        transactionID = 0 

    accountID = item.getAttribute ("accountId")   
    
    # Datum muss passend konvertiert werden
    date = item.getAttribute ("date") 
    formatting = "%Y%m%d"
    datum = datetime.strptime(date, formatting)

    amount = float(item.getAttribute ("amount") )
    TradeID = item.getAttribute ("tradeID")
    if ( TradeID == "" ):
        TradeID = 0 

    balance = float(item.getAttribute ("balance") )
    balance = round ( balance , 2 )

    description = item.getAttribute ("description") 
    activityCode = item.getAttribute ("activityCode") 
    activityDescription = item.getAttribute ("activityDescription") 
  
    #print ( "Transaktion:" , transactionID, " - Account ID",accountID,  " - Datum: " , datum , " - TradeID ", \
    #       TradeID, " - Balance: ", balance, " - Beschreibung " , description , "Code: ", activityCode , " - Desc: ", activityDescription) 

    # In die Datenbank eintragen 
    
    #activityCode  activityDescription
    sql = "INSERT INTO statements (transactionID, accountID, datum , TradeID , amount, balance , description , activityCode, activityDescription) "
    sql += " VALUES (?, ?, ?, ?, ? , ? , ? , ? , ? ) "
    data = (transactionID, accountID, datum , TradeID , amount, balance , description, activityCode , activityDescription)
    
    dbCursor.execute(sql, data)
    # dbConn.commit()


# --------- Start des Programms --------------------------

#Datenbank öffnen 
# Verbindung zur Access-Datenbank herstellen
DBdatenbank = ".\ib.accdb"
DBdriver="{Microsoft Access Driver (*.mdb, *.accdb)}"
print ("Datenbank: ", DBdriver , " ", DBdatenbank )
#dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=", datenbank, ";')
#dbConn = pyodbc.connect(r'Driver=', DBdriver,';DBQ=', DBdatenbank, ';')

# Das andere Wollte nicht 
dbConn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ=.\ib.accdb;')

# Cursor-Objekt erstellen
dbCursor = dbConn.cursor()

#alle Daten löschen 
sql= "DELETE FROM statements"
dbCursor.execute (sql) 


domobject = parse('ib-info-statements.xml')
elementname = 'StatementOfFundsLine'

i = 0 ; 
limit = 100  
for e in domobject.getElementsByTagName( elementname ):
    #Übertragen der Daten in die Datenbank 
    insertStatements ( e , dbCursor, dbConn)

    i = i+1
    if i % limit == 0  :
        print ("")
        print (i, end="")
    print("+", end="")
  
print ("")
print ( "Anzahl Datensätze eingetragen :" , i ) 
      
# Änderungen in der Datenbank speichern
dbConn.commit()
# Verbindung schließen
dbConn.close()


    


Datenbank:  {Microsoft Access Driver (*.mdb, *.accdb)}   .\ib.accdb
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
100++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
200++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
300++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
400++++++++++++++++++++++++++++++++++++++
Anzahl Datensätze eingetragen : 437
